# Introduction to DataFrames
**This is from the excellent Julia tutorial by [Bogumił Kamiński](http://bogumilkaminski.pl/about/)** <br>
Entire tutorial available [here](https://github.com/bkamins/Julia-DataFrames-Tutorial)

In aquaculture applications, one of our most important requirements is integrating data from multiple different sources. Data generated on modern aquaculture farms extend across a wide variety of forms. In situ sensors sample large numbers of environmental variables such as temperature, current veloc-ity, dissolved oxygen (DO), chlorophyll, and salinity. Remote-ly sensed environmental data can sample much larger spatial domains and can be at the bay scale — from land-based sensors such as CODAR-type HF radar — or at the global scale from a satellite-based monitoring system. Informing on farm operations also requires sampling of animal variables such as size, cluster-ing behavior, and movement, and this is typically done using underwater technologies such as video monitoring, hydroacoustic technology, and aerial drone imagery. Further, there are large datasets of pertinent variables that are generated by numerical models such as weather or ocean circulation products. These datasets constitute huge data volumes with distinct characteristics. Integrating and extracting information from these disparate data sources are key to encap-sulating the full dynamics of the farm environment and enabling effective management. Related data from mathematical models are estimates of fish growth and behavior that can be used to guide expected conditions and decision.

#### An effective data science tool demands the ability to consider these disparate datasets and join them in an effective and scalable manner that allows further analysis

In [1]:
using DataFrames

## Joining DataFrames

### Preparing DataFrames for a join

In [2]:
x = DataFrame(ID=[1,2,3,4,missing], name = ["Alice", "Bob", "Conor", "Dave","Zed"])

5×2 DataFrame
 Row │ ID       name   
     │ Int64?   String 
─────┼─────────────────
   1 │       1  Alice
   2 │       2  Bob
   3 │       3  Conor
   4 │       4  Dave
   5 │ missing  Zed

In [3]:
y = DataFrame(id=[1,2,5,6,missing], age=[21,22,23,24,99])

5×2 DataFrame
 Row │ id       age   
     │ Int64?   Int64 
─────┼────────────────
   1 │       1     21
   2 │       2     22
   3 │       5     23
   4 │       6     24
   5 │ missing     99

Rules for the `on` keyword argument:
* a single `Symbol` or string if joining on one column with the same name, e.g. `on=:id`
* a `Pair` of `Symbol`s or string if joining on one column with different names, e.g. `on=:id=>:id2`
* a vector of `Symbol`s or strings if joining on multiple columns with the same name, e.g. `on=[:id1, :id2]`
* a vector of `Pair`s of `Symbol`s or strings if joining on multiple columns with the same name, e.g. `on=[:a1=>:a2, :b1=>:b2]`
* a vector containing a combination of `Symbol`s or strings or `Pair` of `Symbol`s or strings, e.g. `on=[:a1=>:a2, :b1]`

### Standard joins: inner, left, right, outer, semi, anti
The following functions are provided to perform seven kinds of joins:

   - **innerjoin**: the output contains rows for values of the key that exist in all passed data frames.
   - **leftjoin**: the output contains rows for values of the key that exist in the first (left) argument, whether or not that value exists in the second (right) argument.
   - **rightjoin**: the output contains rows for values of the key that exist in the second (right) argument, whether or not that value exists in the first (left) argument.
   - **outerjoin**: the output contains rows for values of the key that exist in any of the passed data frames.
   - **semijoin**: Like an inner join, but output is restricted to columns from the first (left) argument.
   - **antijoin**: The output contains rows for values of the key that exist in the first (left) but not the second (right) argument. As with semijoin, output is restricted to columns from the first (left) argument.
   - **crossjoin**: The output is the cartesian product of rows from all passed data frames.

See the [Wikipedia page on SQL joins](https://en.wikipedia.org/wiki/Join_(SQL)) for more information.

In [12]:
innerjoin(x, y, on=:ID=>:id) # missing is not allowed to join-on by default

LoadError: ArgumentError: missing values in key columns are not allowed when matchmissing == :error

In [13]:
innerjoin(x, y, on=:ID=>:id, matchmissing=:equal)

3×3 DataFrame
 Row │ ID       name    age   
     │ Int64?   String  Int64 
─────┼────────────────────────
   1 │       1  Alice      21
   2 │       2  Bob        22
   3 │ missing  Zed        99

In [14]:
leftjoin(x, y, on="ID"=>"id", matchmissing=:equal)

5×3 DataFrame
 Row │ ID       name    age     
     │ Int64?   String  Int64?  
─────┼──────────────────────────
   1 │       1  Alice        21
   2 │       2  Bob          22
   3 │ missing  Zed          99
   4 │       3  Conor   missing 
   5 │       4  Dave    missing 

In [17]:
rightjoin(x, y, on=:ID=>:id, matchmissing=:equal)

5×3 DataFrame
 Row │ ID       name     age   
     │ Int64?   String?  Int64 
─────┼─────────────────────────
   1 │       1  Alice       21
   2 │       2  Bob         22
   3 │ missing  Zed         99
   4 │       5  missing     23
   5 │       6  missing     24

In [18]:
outerjoin(x, y, on=:ID=>:id, matchmissing=:equal)

7×3 DataFrame
 Row │ ID       name     age     
     │ Int64?   String?  Int64?  
─────┼───────────────────────────
   1 │       1  Alice         21
   2 │       2  Bob           22
   3 │ missing  Zed           99
   4 │       3  Conor    missing 
   5 │       4  Dave     missing 
   6 │       5  missing       23
   7 │       6  missing       24

In [19]:
semijoin(x, y, on=:ID=>:id, matchmissing=:equal)

3×2 DataFrame
 Row │ ID       name   
     │ Int64?   String 
─────┼─────────────────
   1 │       1  Alice
   2 │       2  Bob
   3 │ missing  Zed

In [20]:
antijoin(x, y, on=:ID=>:id, matchmissing=:equal)

2×2 DataFrame
 Row │ ID      name   
     │ Int64?  String 
─────┼────────────────
   1 │      3  Conor
   2 │      4  Dave

### Cross join

(here no `on` argument)

In [21]:
crossjoin(DataFrame(x=[1,2]), DataFrame(y=["a","b","c"]))

6×2 DataFrame
 Row │ x      y      
     │ Int64  String 
─────┼───────────────
   1 │     1  a
   2 │     1  b
   3 │     1  c
   4 │     2  a
   5 │     2  b
   6 │     2  c

### Complex cases of joins

In [22]:
x = DataFrame(id1=[1,1,2,2,missing,missing],
              id2=[1,11,2,21,missing,99],
              name = ["Alice", "Bob", "Conor", "Dave","Zed", "Zoe"])

6×3 DataFrame
 Row │ id1      id2      name   
     │ Int64?   Int64?   String 
─────┼──────────────────────────
   1 │       1        1  Alice
   2 │       1       11  Bob
   3 │       2        2  Conor
   4 │       2       21  Dave
   5 │ missing  missing  Zed
   6 │ missing       99  Zoe

In [23]:
y = DataFrame(id1=[1,1,3,3,missing,missing],
              id2=[11,1,31,3,missing,999],
              age = [21,22,23,24,99, 100])

6×3 DataFrame
 Row │ id1      id2      age   
     │ Int64?   Int64?   Int64 
─────┼─────────────────────────
   1 │       1       11     21
   2 │       1        1     22
   3 │       3       31     23
   4 │       3        3     24
   5 │ missing  missing     99
   6 │ missing      999    100

In [24]:
innerjoin(x, y, on=[:id1, :id2], matchmissing=:equal) # joining on two columns

3×4 DataFrame
 Row │ id1      id2      name    age   
     │ Int64?   Int64?   String  Int64 
─────┼─────────────────────────────────
   1 │       1       11  Bob        21
   2 │       1        1  Alice      22
   3 │ missing  missing  Zed        99

In [25]:
outerjoin(x, y, on=:id1, makeunique=true, indicator=:source, matchmissing=:equal) # with duplicates all combinations are produced

12×6 DataFrame
 Row │ id1      id2      name     id2_1    age      source     
     │ Int64?   Int64?   String?  Int64?   Int64?   String     
─────┼─────────────────────────────────────────────────────────
   1 │       1        1  Alice         11       21  both
   2 │       1       11  Bob           11       21  both
   3 │       1        1  Alice          1       22  both
   4 │       1       11  Bob            1       22  both
   5 │ missing  missing  Zed      missing       99  both
   6 │ missing       99  Zoe      missing       99  both
   7 │ missing  missing  Zed          999      100  both
   8 │ missing       99  Zoe          999      100  both
   9 │       2        2  Conor    missing  missing  left_only
  10 │       2       21  Dave     missing  missing  left_only
  11 │       3  missing  missing       31       23  right_only
  12 │       3  missing  missing        3       24  right_only

In [26]:
# you can force validation of uniqueness of key on which you join
innerjoin(x, y, on=:id1, makeunique=true, validate=(true,true), matchmissing=:equal)

LoadError: ArgumentError: Merge key(s) are not unique in both df1 and df2. df1 contains 3 duplicate keys: NamedTuple{(:id1,), Tuple{Union{Missing, Int64}}}((1,)), ..., NamedTuple{(:id1,), Tuple{Union{Missing, Int64}}}((missing,)). df2 contains 3 duplicate keys: NamedTuple{(:id1,), Tuple{Union{Missing, Int64}}}((1,)), ..., NamedTuple{(:id1,), Tuple{Union{Missing, Int64}}}((missing,)).

mixed `on` argument for joining on multiple columns

In [27]:
x = DataFrame(id1=1:6, id2=[1,2,1,2,1,2], x1 = 'a':'f')

6×3 DataFrame
 Row │ id1    id2    x1   
     │ Int64  Int64  Char 
─────┼────────────────────
   1 │     1      1  a
   2 │     2      2  b
   3 │     3      1  c
   4 │     4      2  d
   5 │     5      1  e
   6 │     6      2  f

In [28]:
y = DataFrame(id1=1:6, ID2=1:6, x2 = 'a':'f')

6×3 DataFrame
 Row │ id1    ID2    x2   
     │ Int64  Int64  Char 
─────┼────────────────────
   1 │     1      1  a
   2 │     2      2  b
   3 │     3      3  c
   4 │     4      4  d
   5 │     5      5  e
   6 │     6      6  f

In [29]:
innerjoin(x, y, on=[:id1, :id2=>:ID2])

2×4 DataFrame
 Row │ id1    id2    x1    x2   
     │ Int64  Int64  Char  Char 
─────┼──────────────────────────
   1 │     1      1  a     a
   2 │     2      2  b     b

joining more than two data frames

In [30]:
xs = [DataFrame("id"=>1:6, "v$i"=>((1:6) .+ 10i)) for i in 1:5]

5-element Vector{DataFrame}:
 6×2 DataFrame
 Row │ id     v1    
     │ Int64  Int64 
─────┼──────────────
   1 │     1     11
   2 │     2     12
   3 │     3     13
   4 │     4     14
   5 │     5     15
   6 │     6     16
 6×2 DataFrame
 Row │ id     v2    
     │ Int64  Int64 
─────┼──────────────
   1 │     1     21
   2 │     2     22
   3 │     3     23
   4 │     4     24
   5 │     5     25
   6 │     6     26
 6×2 DataFrame
 Row │ id     v3    
     │ Int64  Int64 
─────┼──────────────
   1 │     1     31
   2 │     2     32
   3 │     3     33
   4 │     4     34
   5 │     5     35
   6 │     6     36
 6×2 DataFrame
 Row │ id     v4    
     │ Int64  Int64 
─────┼──────────────
   1 │     1     41
   2 │     2     42
   3 │     3     43
   4 │     4     44
   5 │     5     45
   6 │     6     46
 6×2 DataFrame
 Row │ id     v5    
     │ Int64  Int64 
─────┼──────────────
   1 │     1     51
   2 │     2     52
   3 │     3     53
   4 │     4     54
   5 │     5     55
 

In [31]:
innerjoin(xs..., on=:id) # also for outerjoin and crossjoin

6×6 DataFrame
 Row │ id     v1     v2     v3     v4     v5    
     │ Int64  Int64  Int64  Int64  Int64  Int64 
─────┼──────────────────────────────────────────
   1 │     1     11     21     31     41     51
   2 │     2     12     22     32     42     52
   3 │     3     13     23     33     43     53
   4 │     4     14     24     34     44     54
   5 │     5     15     25     35     45     55
   6 │     6     16     26     36     46     56